### Imports

In [69]:
import pandas as pd
import os
import pprint

### View data

In [70]:
file_path = "../raw data"
csv_files = os.listdir(file_path)

In [71]:
%%time
df_list = []
standard_format_files = set([#'whistler.csv'
                             'squaw.csv'
                             ,'snowbird.csv'
                             ,'mtBaker.csv'
                             ,'jackson.csv'
                             ,'bigsky.csv'])
for csv_file in csv_files:
    if csv_file in standard_format_files:
        df_ = pd.read_csv(file_path+"/"+csv_file,parse_dates={'date':[0]}, infer_datetime_format=True)
        df_list.append(df_)
snow_data = pd.concat(df_list)
#snow_data.fillna(0,inplace=True)
display(snow_data.head())

,date,mountain,max
0,1992-01-01,Big Sky,16.5
1,1993-01-01,Big Sky,21.4
2,1994-01-01,Big Sky,13.8
3,1995-01-01,Big Sky,28.6
4,1996-01-01,Big Sky,30.7


CPU times: user 22.2 ms, sys: 3.18 ms, total: 25.4 ms
Wall time: 24 ms


### Check for Null Values

In [72]:
nulls = snow_data[snow_data.isnull().any(axis=1)]
print("Total rows w/ null values: {:,}".format(nulls.shape[0]))
nulls.head()

Total rows w/ null values: 1


,date,mountain,max
26,2016-01-01,snowbird,NaN


### Data cleanse 

In [73]:
"date" in snow_data.columns

True

In [74]:
%%time
###
### unify timestamp
###
snow_data.loc[:,"day"] = pd.to_datetime(snow_data.date.dt.strftime('%Y-%m-%d'), format='%Y-%m-%d')
###
### review
###
print("REVIEW:")
display(snow_data.loc[snow_data.loc[:,"mountain"]=="whistler",].head())

REVIEW:


,date,mountain,max,day


CPU times: user 10.8 ms, sys: 2.74 ms, total: 13.5 ms
Wall time: 11.3 ms


### group by day

In [75]:
###
### group by day
###
print("GROUP BY DAY:")
snow_data_g = snow_data.groupby(by=["day"]).agg({"mountain":["unique","nunique"],"max":"unique"})
display(snow_data_g.head())

GROUP BY DAY:


mountain              max
             unique nunique   unique
day                                 
1981-01-01  [squaw]       1   [38.4]
1982-01-01  [squaw]       1   [92.1]
1983-01-01  [squaw]       1  [110.2]
1984-01-01  [squaw]       1   [49.6]
1985-01-01  [squaw]       1   [33.1]

### create data container

In [76]:
%%time
###
### create data container
###
unique_mtn_values = snow_data.mountain.unique().tolist()
data = {mtn:[] for i,mtn in enumerate(unique_mtn_values)}
data["date"] = []
display(data)


{'Big Sky': [],
 'squaw': [],
 'Mt. Baker': [],
 'Jackson Hole': [],
 'snowbird': [],
 'date': []}

CPU times: user 1.97 ms, sys: 1.29 ms, total: 3.25 ms
Wall time: 2.05 ms


### finalize data format for final df

In [77]:
###
### finalize data format for final df
### 
def fill_in_missing_snow_values(row, data,unique_mtn_values):
    d = dict(zip(row.mountain["unique"],row["max"]["unique"]))
    for mtn in unique_mtn_values:
        if mtn in d:
            data[mtn].append(d[mtn])
        else:
            data[mtn].append(None)
    data["date"].append(row.name)
    
snow_data_g.apply(fill_in_missing_snow_values,axis=1,args=[data,unique_mtn_values])
final_struct = pd.DataFrame(data)
final_struct.set_index("date",inplace=True)
print("FINAL DATAFRAME STRUCTURE:")
display(final_struct.head())

FINAL DATAFRAME STRUCTURE:


,Big Sky,squaw,Mt. Baker,Jackson Hole,snowbird
date,,,,,
1981-01-01,NaN,38.4,NaN,NaN,NaN
1982-01-01,NaN,92.1,NaN,NaN,NaN
1983-01-01,NaN,110.2,NaN,NaN,NaN
1984-01-01,NaN,49.6,NaN,NaN,NaN
1985-01-01,NaN,33.1,NaN,NaN,NaN


### fill in missing dates

See [Date offsets](http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

In [78]:
###
### fill in missing dates
### 
def custom_resampler(array):
    try: 
        value = sum(array)
    except TypeError:
        value = None
    return value
final_df = final_struct.resample('A').apply(custom_resampler)
print("FiNAL DATAFRAME:")
display(final_df)

FiNAL DATAFRAME:


,Big Sky,squaw,Mt. Baker,Jackson Hole,snowbird
date,,,,,
1981-12-31,NaN,38.4,NaN,NaN,NaN
1982-12-31,NaN,92.1,NaN,NaN,NaN
1983-12-31,NaN,110.2,NaN,NaN,NaN
1984-12-31,NaN,49.6,NaN,NaN,NaN
1985-12-31,NaN,33.1,NaN,NaN,NaN
1986-12-31,NaN,70.5,NaN,NaN,NaN
1987-12-31,NaN,33.3,NaN,17.3,NaN
1988-12-31,NaN,29.7,NaN,23.8,NaN
1989-12-31,NaN,54.5,NaN,38.7,NaN


### Munge & save D3js data

In [82]:
d3_df = final_df.reset_index()
d3_data_format = d3_df.to_json(orient='records',date_format="iso")
d3_df.to_json("../d3js_data/d3js_data_yearEnd(sansWhistler).json",orient='records',date_format="iso")
pprint.pprint(d3_data_format)

('[{"date":"1981-12-31T00:00:00.000Z","Big Sky":null,"squaw":38.4,"Mt. '
 'Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1982-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":92.1,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1983-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":110.2,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1984-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":49.6,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1985-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":33.1,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1986-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":70.5,"Mt. Baker":null,"Jackson '
 'Hole":null,"snowbird":null},{"date":"1987-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":33.3,"Mt. Baker":null,"Jackson '
 'Hole":17.3,"snowbird":null},{"date":"1988-12-31T00:00:00.000Z","Big '
 'Sky":null,"squaw":29.7,"Mt. Baker":null,"Jackson '
 'Hole":23.8,"snowbird":nu

In [80]:
d3_df.describe()

,Big Sky,squaw,Mt. Baker,Jackson Hole,snowbird
count,27.000000,38.000000,15.000000,32.000000,27.000000
mean,22.211111,55.915789,67.833333,28.743750,46.103704
std,5.765036,26.032220,20.004273,8.100017,15.191862
min,13.800000,18.400000,22.600000,17.300000,21.600000
25%,17.900000,34.050000,59.700000,22.900000,34.500000
50%,20.800000,49.750000,72.300000,25.450000,42.900000
75%,26.150000,74.075000,80.500000,36.250000,58.050000
max,33.400000,116.500000,97.400000,44.300000,75.100000


In [81]:
d3_df.loc[d3_df.loc[:,"Jackson Hole"]>0,]

,date,Big Sky,squaw,Mt. Baker,Jackson Hole,snowbird
6,1987-12-31,NaN,33.3,NaN,17.3,NaN
7,1988-12-31,NaN,29.7,NaN,23.8,NaN
8,1989-12-31,NaN,54.5,NaN,38.7,NaN
9,1990-12-31,NaN,27.9,NaN,23.1,34.0
10,1991-12-31,NaN,33.2,NaN,29.0,40.0
11,1992-12-31,16.5,33.7,NaN,19.0,23.0
12,1993-12-31,21.4,99.9,NaN,34.6,66.9
13,1994-12-31,13.8,31.9,NaN,21.5,NaN
14,1995-12-31,28.6,116.5,NaN,33.9,58.9
15,1996-12-31,30.7,65.1,NaN,38.5,52.9
